In [2]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
with open('kakaoapi.txt', mode='r') as key_file:
    kakao_key = key_file.read()

In [4]:
# 데이터 로드
park = pd.read_csv('서울시 주요 공원현황.csv', encoding='euc-kr')
park.shape, park.head(2)

((132, 20),
    공원번호       공원명                                               공원개요  \
 0     1  남산도시자연공원  남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...   
 1     2    길동생태공원  길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...   
 
                                              면적        개원일  \
 0  2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡  1968.9.10   
 1                                        80683㎡  1999.5.20   
 
                                                 주요시설  \
 0  기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...   
 1               탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터   
 
                                         주요식물  \
 0          소나무 단풍 아카시아 상수리나무 등 191종 2881870주   
 1  소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본   
 
                                                  안내도  \
 0  http://parks.seoul.go.kr/upload/seditorMulti/2...   
 1  http://parks.seoul.go.kr/template/common/img/p...   
 
                                                 오시는길  \
 0  명동역(남산케이블카 와룡묘 서울애니메

In [53]:
park.columns

Index(['공원번호', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [5]:
# 불필요한 칼럼 제거
park.drop(columns=['공원번호', '공원개요', '개원일', '주요시설', '주요식물',
                '안내도', '오시는길', '이용시참고사항', '이미지', '지역', '관리부서', '전화번호',
                'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'], inplace=True,  axis=1,)
park.head(2)

,공원명,면적,공원주소
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동)
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원)


In [74]:
park.columns

Index(['공원명', '면적', '공원주소'], dtype='object')

In [6]:
# 널 값 제거
park=park.dropna(axis=0)
park

,공원명,면적,공원주소
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,서울특별시 중구 삼일대로 231(예장동)
1,길동생태공원,80683㎡,서울특별시 강동구 천호대로 1291(길동생태공원)
2,서울대공원,9132690m²,경기도 과천시 대공원광장로 102
3,서울숲,480994㎡,서울특별시 성동구 뚝섬로 273 (성수동1가)
4,월드컵공원,2284085㎡,서울특별시 마포구 하늘공원로 84(월드컵공원)
...,...,...,...
126,서일대뒷산공원,631021㎡,서울특별시 중랑구 면목동 산19
127,용마도시자연공원(사가정공원),109635㎡,서울특별시 중랑구 면목7동 산50
128,문화비축기지,140022㎡,서울특별시 마포구 증산로 87
129,경춘선숲길,211392㎡,노원구 공릉2동 산 82-2


In [7]:
# addr -> str 으로 변환
addr = park['공원주소'].str.replace(',','')[0]
addr

'서울특별시 중구 삼일대로 231(예장동)'

In [66]:
type(addr)

str

In [10]:
# 예외 처리(try-except)
import requests
lng_list, lat_list = [], []
for i in park.index:
    try:
        url = f"{local_url}?query={quote(park['공원주소'][i])}"  # 주소의 i번째
        result = requests.get(url, headers={"Authorization" : f"KakaoAK {kakao_key}"}).json()
        lng = result['documents'][0]['x']
        lat = result['documents'][0]['y']
        lng_list.append(lng)
        lat_list.append(lat)
    except:
        print(park['공원명'][i], park['공원주소'][i])

남산도시자연공원 서울특별시 중구 삼일대로 231(예장동)
길동생태공원 서울특별시 강동구 천호대로 1291(길동생태공원) 
서울대공원 경기도 과천시 대공원광장로 102
서울숲 서울특별시 성동구 뚝섬로 273 (성수동1가) 
월드컵공원 서울특별시 마포구 하늘공원로 84(월드컵공원)
광화문시민열린마당 서울특별시 종로구 세종로 76-2
진관근린공원(구파발폭포) 서울특별시 은평구 진관동 393
중마루근린공원 서울특별시 영등포구 영등포동2가 222
응봉공원 서울특별시 성동구 금호동1가 산1-72
매화근린공원 서울특별시 강서구 등촌동 630-14
성수근린공원 서울특별시 성동구 성수2가 3동 302-25
샛마을근린공원 서울특별시 강동구 명일1동 288
여의도근린공원 서울특별시 영등포구 여의공원로 68 (여의도동)
간데메근린공원 서울특별시 동대문구 서울시립대로2길 59(간데메공원)
훈련원근린공원 서울특별시 중구 을지로 227
영등포근린공원 서울특별시 영등포구 신길로 275
보라매근린공원 서울특별시 동작구 여의대방로20길 33(보라매공원)
어린이대공원 서울특별시 광진구 능동로 216(어린이대공원)
천호근린공원 서울특별시 강동구 올림픽로 702(천호공원)
용산가족공원 서울특별시 용산구 서빙고로 185
독립근린공원 서울특별시 서대문구 통일로251(독립공원)
낙산근린공원 서울특별시 종로구 낙산길 41 (우) 03085
양재시민의숲 서울특별시 서초구 매헌로 99 (양재동)
관악산도시자연공원 서울특별시 관악구 관악로 1(관악산공원)
효창근린공원 서울특별시 용산구 효창원로 177-18
감로천생태공원(관악산) 서울특별시 금천구 독산4동 371?375 373일대
개운산근린공원 서울특별시 성북구 북악산로 949-73(돈암동 6-1)
개웅산근린공원 서울특별시 구로구 오류동 산13-1
경희궁 서울특별시 종로구 새문안로 55(신문로2가 2-1)
국립현충원 서울특별시 동작구 사당2동 산41-2
계남근린공원 서울특별시 양천구 신정동 산44-2일대
고척근린공원 서울특별시 구로구 고척로45길 39(고척공원

In [ ]:
import requests
lng_list, lat_list = [], []
for i in park.index:
    try:
        url = f"{local_url}?query={quote(park['공원주소'][i])}"  # 주소의 i번째
        result = requests.get(url, headers={"Authorization" : f"KakaoAK {kakao_key}"}).json()
        lng = result['documents'][0]['x']
        lat = result['documents'][0]['y']
        lng_list.append(lng)
        lat_list.append(lat)
    except:
        print(park['공원명'][i], park['공원주소'][i])

In [86]:
park['위도'] = lat_list
park['경도'] = lng_list
park

ValueError: Length of values (107) does not match length of index (129)